# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [248]:
problem = '''You are in charge of the supply purchasing of a company that produces two kinds of drugs. 
The drugs contain a specific active agent, which is extracted from two different kinds of raw materials that should be purchased on the market. 

The goal is to maximize the total profit obtained from producing the drugs, which means minimizing purchasing costs for buying the raw materials and minimizing operational costs for producing the drugs while maximizing sales. 

You are given a budget for purchasing raw materials and operating the production process of the drugs which cannot be exceeded. 
Additionally, you need to keep the capacity constraints for the production of the drugs in mind as there is only a limited amount of manpower as well as equipment hours available. 
Your company can also only store a limited amount of raw materials. 
Most importantly, the amount of active ingredient extracted from the raw materials you purchase needs to satisfy the required amount of active ingredient in the drugs your company produces. 
Importantly, your experience with the production of the drugs has shown the the amount of active ingredient your process extracts from the two raw materials can vary by up to 0.5% and 2%, respectively. 
During purchasing, you need to make sure that no matter how much the amount of active ingredient varies, the required amount for the drug production is always met. 
'''

## 2. Generate the mathematical model

In [249]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [250]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Drug Production

### Sets and Indices:

* $i \in I$: Set of raw materials (i.e., $I = \{1, 2\}$)
* $j \in J$: Set of drugs (i.e., $J = \{1, 2\}$)

### Parameters:

* $c_i$: Cost per unit of raw material $i$
* $p_j$: Selling price per unit of drug $j$
* $o_j$: Operational cost per unit of drug $j$
* $a_{ij}$: Nominal amount of active ingredient extracted from one unit of raw material $i$ for drug $j$
* $\alpha_i$: Variability percentage of active ingredient extraction from raw material $i$ (i.e., $\alpha_1 = 0.005$, $\alpha_2 = 0.02$)
* $r_j$: Required amount of active ingredient per unit of drug $j$
* $B$: Budget for purchasing and operations
* $C_i$: Storage capacity for raw material $i$
* $M$: Available manpower hours
* $E$: Available equipment hours
* $m_j$: Manpower hours required per unit of drug $j$
* $e_j$: Equipment hours required per unit of drug $j$

### Decision Variables:

* $x_i$: Quantity of raw material $i$ to purchase
* $y_j$: Quantity of drug $j$ to produce

### Objective Function (Maximize Profit):

Maximize $Z = \sum_{j \in J} (p_j - o_j)y_j - \sum_{i \in I} c_ix_i$

### Constraints:

1. **Budget Constraint:** $\sum_{i \in I} c_ix_i + \sum_{j \in J} o_jy_j \leq B$
2. **Storage Capacity:** $x_i \leq C_i$ for all $i \in I$
3. **Manpower Capacity:** $\sum_{j \in J} m_jy_j \leq M$
4. **Equipment Capacity:** $\sum_{j \in J} e_jy_j \leq E$
5. **Active Ingredient Requirement (Worst-Case Scenario):** 
    * For each drug $j$:
        * $\sum_{i \in I} (1 - \alpha_i) a_{ij} x_i \geq r_j y_j$
6. **Non-negativity:** $x_i, y_j \geq 0$ for all $i \in I, j \in J$

### Explanation:

* The objective function aims to maximize the total profit, which is the difference between revenue from selling drugs and the cost of purchasing raw materials and operating the production process.
* Constraint 1 ensures that the total cost of purchasing and operations remains within the budget.
* Constraints 2, 3, and 4 enforce the storage, manpower, and equipment capacity limitations, respectively.
* Constraint 5 is the key constraint that guarantees sufficient active ingredient for drug production even in the worst-case scenario where the extraction rate deviates negatively due to variability. 
* Constraint 6 ensures that the quantity of raw materials and drugs cannot be negative. 

### Solving the Model:

This model is a linear programming (LP) problem and can be solved efficiently using various optimization software packages or solvers like CPLEX, Gurobi, or even Excel Solver. The solution will provide the optimal quantities of each raw material to purchase and each drug to produce to maximize the company's profit while respecting all the constraints. 


## 3. Generate the pyomo code

In [251]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [252]:
#Showing the code in a formatted way
Markdown(response2.text)

## Pyomo Implementation of Drug Production Model

```python
from pyomo.environ import *

# Sample data (replace with actual data)
c = {1: 10, 2: 15}  # Cost per unit of raw material
p = {1: 50, 2: 70}  # Selling price per unit of drug
o = {1: 5, 2: 8}   # Operational cost per unit of drug
a = {
    (1, 1): 0.8,
    (1, 2): 0.6,
    (2, 1): 0.7,
    (2, 2): 0.9
}  # Nominal amount of active ingredient
alpha = {1: 0.005, 2: 0.02}  # Variability percentage
r = {1: 0.5, 2: 0.7}  # Required active ingredient per unit
B = 10000  # Budget
C = {1: 500, 2: 600}  # Storage capacity
M = 2000  # Available manpower hours
E = 1500  # Available equipment hours
m = {1: 2, 2: 3}  # Manpower hours per unit
e = {1: 1.5, 2: 2.5}  # Equipment hours per unit

# Model creation
model = ConcreteModel()

# Sets
model.I = Set(initialize=c.keys())  # Raw materials
model.J = Set(initialize=p.keys())  # Drugs

# Parameters
model.c = Param(model.I, initialize=c)
model.p = Param(model.J, initialize=p)
model.o = Param(model.J, initialize=o)
model.a = Param(model.I, model.J, initialize=a)
model.alpha = Param(model.I, initialize=alpha)
model.r = Param(model.J, initialize=r)
model.B = Param(initialize=B)
model.C = Param(model.I, initialize=C)
model.M = Param(initialize=M)
model.E = Param(initialize=E)
model.m = Param(model.J, initialize=m)
model.e = Param(model.J, initialize=e)

# Decision variables
model.x = Var(model.I, domain=NonNegativeReals)
model.y = Var(model.J, domain=NonNegativeReals)

# Objective function
model.profit = Objective(
    expr=sum((model.p[j] - model.o[j]) * model.y[j] for j in model.J) -
    sum(model.c[i] * model.x[i] for i in model.I),
    sense=maximize
)

# Constraints
model.budget_constraint = Constraint(
    expr=sum(model.c[i] * model.x[i] for i in model.I) +
    sum(model.o[j] * model.y[j] for j in model.J) <= model.B
)
model.storage_constraint = Constraint(model.I, rule=lambda model, i: model.x[i] <= model.C[i])
model.manpower_constraint = Constraint(expr=sum(model.m[j] * model.y[j] for j in model.J) <= model.M)
model.equipment_constraint = Constraint(expr=sum(model.e[j] * model.y[j] for j in model.J) <= model.E)
model.active_ingredient_constraint = Constraint(model.J, rule=lambda model, j:
                                               sum((1 - model.alpha[i]) * model.a[i, j] * model.x[i] for i in model.I) >= model.r[j] * model.y[j])

# Solve the model
solver = SolverFactory('glpk')  # You can change the solver as needed
results = solver.solve(model)

# Print results (you might want to improve formatting)
print(results)

# Accessing results
print("Optimal profit:", model.profit())
for i in model.I:
    print(f"Quantity of raw material {i} to purchase: {model.x[i]():.2f}")
for j in model.J:
    print(f"Quantity of drug {j} to produce: {model.y[j]():.2f}")
``` 


## 4. Input problem data and try running the generated code

In [266]:
from pyomo.environ import *

# Sample data (replace with actual data)
c = {1: 100, 2: 199.90}  # Cost per unit of raw material
p = {1: 6200, 2: 6900}  # Selling price per unit of drug
o = {1: 700, 2: 800}   # Operational cost per unit of drug
a = {
    (1, 1): 0.01,
    (1, 2): 0.01,
    (2, 1): 0.02,
    (2, 2): 0.02
}  # Nominal amount of active ingredient
alpha = {1: 0.005, 2: 0.02}  # Variability percentage
r = {1: 0.5, 2: 0.6}  # Required active ingredient per unit
B = 100000  # Budget
C = {1: 5000, 2: 5000}  # Storage capacity
M = 2000  # Available manpower hours
E = 800  # Available equipment hours
m = {1: 90, 2: 100}  # Manpower hours per unit
e = {1: 40, 2: 50}  # Equipment hours per unit

# Model creation
model = ConcreteModel()

# Sets
model.I = Set(initialize=c.keys())  # Raw materials
model.J = Set(initialize=p.keys())  # Drugs

# Parameters
model.c = Param(model.I, initialize=c)
model.p = Param(model.J, initialize=p)
model.o = Param(model.J, initialize=o)
model.a = Param(model.I, model.J, initialize=a)
model.alpha = Param(model.I, initialize=alpha)
model.r = Param(model.J, initialize=r)
model.B = Param(initialize=B)
model.C = Param(model.I, initialize=C)
model.M = Param(initialize=M)
model.E = Param(initialize=E)
model.m = Param(model.J, initialize=m)
model.e = Param(model.J, initialize=e)

# Decision variables
model.x = Var(model.I, domain=NonNegativeReals)
model.y = Var(model.J, domain=NonNegativeReals)

# Objective function
model.profit = Objective(
    expr=sum((model.p[j] - model.o[j]) * model.y[j] for j in model.J) -
    sum(model.c[i] * model.x[i] for i in model.I),
    sense=maximize
)

# Constraints
model.budget_constraint = Constraint(
    expr=sum(model.c[i] * model.x[i] for i in model.I) +
    sum(model.o[j] * model.y[j] for j in model.J) <= model.B
)
model.storage_constraint = Constraint(model.I, rule=lambda model, i: model.x[i] <= model.C[i])
model.manpower_constraint = Constraint(expr=sum(model.m[j] * model.y[j] for j in model.J) <= model.M)
model.equipment_constraint = Constraint(expr=sum(model.e[j] * model.y[j] for j in model.J) <= model.E)
model.active_ingredient_constraint = Constraint(model.J, rule=lambda model, j:
                                               sum((1 - model.alpha[i]) * model.a[i, j] * model.x[i] for i in model.I) >= model.r[j] * model.y[j])

# Solve the model
solver = SolverFactory('glpk')  # You can change the solver as needed
results = solver.solve(model)

# Print results (you might want to improve formatting)
print(results)

# Accessing results
print("Optimal profit:", model.profit())
for i in model.I:
    print(f"Quantity of raw material {i} to purchase: {model.x[i]():.2f}")
for j in model.J:
    print(f"Quantity of drug {j} to produce: {model.y[j]():.2f}")


Problem: 
- Name: unknown
  Lower bound: 54447.7489488258
  Upper bound: 54447.7489488258
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 4
  Number of nonzeros: 16
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04232978820800781
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Optimal profit: 54447.748948825814
Quantity of raw material 1 to purchase: 492.26
Quantity of raw material 2 to purchase: 0.00
Quantity of drug 1 to produce: 9.80
Quantity of drug 2 to produce: 8.16


## 5. Correct the code to verify model viability (optional)

In [265]:
from pyomo.environ import *

# Sample data (replace with actual data)
c = {1: 100, 2: 199.90}  # Cost per unit of raw material
p = {1: 6200, 2: 6900}  # Selling price per unit of drug
o = {1: 700, 2: 800}   # Operational cost per unit of drug
a = {
    (1, 1): 0.01,
    (1, 2): 0.01,
    (2, 1): 0.02,
    (2, 2): 0.02
}  # Nominal amount of active ingredient
alpha = {1: 0.005, 2: 0.02}  # Variability percentage
r = {1: 0.5, 2: 0.6}  # Required active ingredient per unit
B = 100000  # Budget
C = 1000  # Storage capacity #ALTERED
M = 2000  # Available manpower hours
E = 800  # Available equipment hours
m = {1: 90, 2: 100}  # Manpower hours per unit
e = {1: 40, 2: 50}  # Equipment hours per unit

# Model creation
model = ConcreteModel()

# Sets
model.I = Set(initialize=c.keys())  # Raw materials
model.J = Set(initialize=p.keys())  # Drugs

# Parameters
model.c = Param(model.I, initialize=c)
model.p = Param(model.J, initialize=p)
model.o = Param(model.J, initialize=o)
model.a = Param(model.I, model.J, initialize=a)
model.alpha = Param(model.I, initialize=alpha)
model.r = Param(model.J, initialize=r)
model.B = Param(initialize=B)
#model.C = Param(model.I, initialize=C)
model.M = Param(initialize=M)
model.E = Param(initialize=E)
model.m = Param(model.J, initialize=m)
model.e = Param(model.J, initialize=e)

# Decision variables
model.x = Var(model.I, domain=NonNegativeReals)
model.y = Var(model.J, domain=NonNegativeReals)

# Objective function
model.profit = Objective(
    expr=sum((model.p[j] - model.o[j]) * model.y[j] for j in model.J) -
    sum(model.c[i] * model.x[i] for i in model.I),
    sense=maximize
)

# Constraints
model.budget_constraint = Constraint(
    expr=sum(model.c[i] * model.x[i] for i in model.I) +
    sum(model.o[j] * model.y[j] for j in model.J) <= model.B
)
model.storage_constraint = Constraint(expr=sum(model.x[i] for i in model.I)<=C)
model.manpower_constraint = Constraint(expr=sum(model.m[j] * model.y[j] for j in model.J) <= model.M)
model.equipment_constraint = Constraint(expr=sum(model.e[j] * model.y[j] for j in model.J) <= model.E)
model.active_ingredient_constraint = Constraint(expr=(0.01*0.995*model.x[1] + 0.02*0.98*model.x[2] - 0.5*model.y[1] - 0.6* model.y[2]>=0))

# Solve the model
solver = SolverFactory('glpk')  # You can change the solver as needed
results = solver.solve(model)

# Print results (you might want to improve formatting)
print(results)

# Accessing results
print("Optimal profit:", model.profit())
for i in model.I:
    print(f"Quantity of raw material {i} to purchase: {model.x[i]():.2f}")
for j in model.J:
    print(f"Quantity of drug {j} to produce: {model.y[j]():.2f}")


Problem: 
- Name: unknown
  Lower bound: 8294.56683928728
  Upper bound: 8294.56683928728
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 4
  Number of nonzeros: 14
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.03960728645324707
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Optimal profit: 8294.566839287349
Quantity of raw material 1 to purchase: 877.73
Quantity of raw material 2 to purchase: 0.00
Quantity of drug 1 to produce: 17.47
Quantity of drug 2 to produce: 0.00


## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [267]:
print(response.text)

## Mathematical Optimization Model for Drug Production

### Sets and Indices:

* $i \in I$: Set of raw materials (i.e., $I = \{1, 2\}$)
* $j \in J$: Set of drugs (i.e., $J = \{1, 2\}$)

### Parameters:

* $c_i$: Cost per unit of raw material $i$
* $p_j$: Selling price per unit of drug $j$
* $o_j$: Operational cost per unit of drug $j$
* $a_{ij}$: Nominal amount of active ingredient extracted from one unit of raw material $i$ for drug $j$
* $\alpha_i$: Variability percentage of active ingredient extraction from raw material $i$ (i.e., $\alpha_1 = 0.005$, $\alpha_2 = 0.02$)
* $r_j$: Required amount of active ingredient per unit of drug $j$
* $B$: Budget for purchasing and operations
* $C_i$: Storage capacity for raw material $i$
* $M$: Available manpower hours
* $E$: Available equipment hours
* $m_j$: Manpower hours required per unit of drug $j$
* $e_j$: Equipment hours required per unit of drug $j$

### Decision Variables:

* $x_i$: Quantity of raw material $i$ to purchase
* $y_j$: Quan

In [268]:
print(response2.text)

## Pyomo Implementation of Drug Production Model

```python
from pyomo.environ import *

# Sample data (replace with actual data)
c = {1: 10, 2: 15}  # Cost per unit of raw material
p = {1: 50, 2: 70}  # Selling price per unit of drug
o = {1: 5, 2: 8}   # Operational cost per unit of drug
a = {
    (1, 1): 0.8,
    (1, 2): 0.6,
    (2, 1): 0.7,
    (2, 2): 0.9
}  # Nominal amount of active ingredient
alpha = {1: 0.005, 2: 0.02}  # Variability percentage
r = {1: 0.5, 2: 0.7}  # Required active ingredient per unit
B = 10000  # Budget
C = {1: 500, 2: 600}  # Storage capacity
M = 2000  # Available manpower hours
E = 1500  # Available equipment hours
m = {1: 2, 2: 3}  # Manpower hours per unit
e = {1: 1.5, 2: 2.5}  # Equipment hours per unit

# Model creation
model = ConcreteModel()

# Sets
model.I = Set(initialize=c.keys())  # Raw materials
model.J = Set(initialize=p.keys())  # Drugs

# Parameters
model.c = Param(model.I, initialize=c)
model.p = Param(model.J, initialize=p)
model.o = Param